In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, confusion_matrix, classification_report

In [2]:
df = pd.read_csv('HouseTS.csv')

In [3]:
df[['zipcode', 'date']]

,zipcode,date
0,30002,2012-03-31
1,30002,2012-04-30
2,30002,2012-05-31
3,30002,2012-06-30
4,30002,2012-07-31
...,...,...
884087,34698,2023-08-31
884088,34698,2023-09-30
884089,34698,2023-10-31
884090,34698,2023-11-30


In [4]:
df_sorted = df.sort_values(['zipcode', 'date']).reset_index(drop=True)
df_sorted[['zipcode', 'date']]

,zipcode,date
0,1431,2012-03-31
1,1431,2012-04-30
2,1431,2012-05-31
3,1431,2012-06-30
4,1431,2012-07-31
...,...,...
884087,98686,2023-08-31
884088,98686,2023-09-30
884089,98686,2023-10-31
884090,98686,2023-11-30


In [5]:
type(df_sorted['date'][0]), df_sorted['date'][0]

(str, '2012-03-31')

In [6]:
df_sorted['date'] = pd.to_datetime(df_sorted['date'])
df_sorted['date']

0        2012-03-31
1        2012-04-30
2        2012-05-31
3        2012-06-30
4        2012-07-31
            ...    
884087   2023-08-31
884088   2023-09-30
884089   2023-10-31
884090   2023-11-30
884091   2023-12-31
Name: date, Length: 884092, dtype: datetime64[ns]

In [7]:
df_grouped = df_sorted.groupby('zipcode')

In [8]:
ts_all = pd.DataFrame()

ls = [1,3,6,12]

i = 1

#for i in ls:
ts_all[f'target_profitable_{i}m'] = (df_grouped['price'].shift(-i) > 1.02 * df_sorted['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_all['zipcode_obj'] = df_grouped['zipcode'].shift(0).astype('object') # but remains still an integer...
ts_all['date'] = df_sorted['date']
numeric_column_names = df_sorted.select_dtypes(include='number').columns # drop the lags of zipcode-> this in Jessica's code.


for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_all[f'{feature}_{lag}m'] = df_grouped[f'{feature}'].shift(lag)

ts_all = ts_all.drop(columns=['zipcode_1m', 'zipcode_2m', 'zipcode_3m'])

/tmp/ipykernel_110428/2526076371.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_all[f'{feature}_{lag}m'] = df_grouped[f'{feature}'].shift(lag)
/tmp/ipykernel_110428/2526076371.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_all[f'{feature}_{lag}m'] = df_grouped[f'{feature}'].shift(lag)
/tmp/ipykernel_110428/2526076371.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

In [9]:
#type(ts_all)
ts_all = ts_all.dropna()
ts_all

,target_profitable_1m,zipcode_obj,date,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
3,0,1431,2012-06-30,177500.0,124000.0,89000.0,207450.0,229500.0,252250.0,120.984975,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,194580.490265,194446.137500,195148.086474
4,0,1431,2012-07-31,217000.0,177500.0,124000.0,203750.0,207450.0,229500.0,121.340580,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,195081.437213,194580.490265,194446.137500
5,0,1431,2012-08-31,130000.0,217000.0,177500.0,193900.0,203750.0,207450.0,81.948640,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,196403.714599,195081.437213,194580.490265
6,0,1431,2012-09-30,130000.0,130000.0,217000.0,203700.0,193900.0,203750.0,95.205795,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,197743.074185,196403.714599,195081.437213
7,0,1431,2012-10-31,191000.0,130000.0,130000.0,162537.5,203700.0,193900.0,118.367237,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,198843.413326,197743.074185,196403.714599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884087,0,98686,2023-08-31,515000.0,530900.0,523227.5,527450.0,520000.0,474900.0,305.216752,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,561259.870706,558047.224949,555511.734585
884088,0,98686,2023-09-30,502900.0,515000.0,530900.0,550000.0,527450.0,520000.0,305.790501,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,564276.110320,561259.870706,558047.224949
884089,0,98686,2023-10-31,509900.0,502900.0,515000.0,511755.0,550000.0,527450.0,311.643836,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,566663.763169,564276.110320,561259.870706
884090,0,98686,2023-11-30,525000.0,509900.0,502900.0,504450.0,511755.0,550000.0,306.527550,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,568266.780424,566663.763169,564276.110320


In [10]:
type(df_sorted['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
df_sorted[['zipcode','date']]

,zipcode,date
0,1431,2012-03-31
1,1431,2012-04-30
2,1431,2012-05-31
3,1431,2012-06-30
4,1431,2012-07-31
...,...,...
884087,98686,2023-08-31
884088,98686,2023-09-30
884089,98686,2023-10-31
884090,98686,2023-11-30


In [12]:
def split_train_predict(df):
    # Ensure it's sorted by zipcode and date
    df = df.sort_values(['zipcode_obj', 'date'])

    # Group by zipcode
    train_list = []
    predict_list = []

    for zipcode, group in df.groupby('zipcode_obj'):
        train_part = group.iloc[:-1]
        predict_part = group.iloc[-1:]

        train_list.append(train_part)
        predict_list.append(predict_part)

    # Concatenate the results
    df_train = pd.concat(train_list).reset_index(drop=True)
    df_predict = pd.concat(predict_list).reset_index(drop=True)

    return df_train, df_predict

# Apply the split
df_train, df_predict = split_train_predict(ts_all)

# Optional: verify
print("Train set:", df_train.shape, 'Type', type(df_train))
print("Predict set:", df_predict.shape, 'Type', type(df_predict))

Train set: (859188, 108) Type <class 'pandas.core.frame.DataFrame'>
Predict set: (6226, 108) Type <class 'pandas.core.frame.DataFrame'>


In [13]:
# Load your data, step 1
df = df_train

# Step 2: Define target and features
target = 'target_profitable_1m'
y = df[target]
X = df.drop(columns=[target])

# Step 3: Check class distribution
print("Class distribution in full dataset:")
print(y.value_counts())


Class distribution in full dataset:
target_profitable_1m
0    820434
1     38754
Name: count, dtype: int64


In [14]:
df_predict=df_predict.drop(columns='target_profitable_1m')


In [15]:
df_predict

,zipcode_obj,date,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,median_ppsf_2m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,1431,2023-12-31,469000.0,462500.0,465000.0,398888.0,450000.0,462450.0,200.601023,219.758478,...,3170.0,3170.0,3170.0,3170.0,1541.0,1541.0,1541.0,431098.634112,429997.675464,427411.135867
1,1432,2023-12-31,476000.0,485000.0,465000.0,499900.0,535000.0,507450.0,343.023256,323.688969,...,7887.0,7887.0,7887.0,7887.0,3905.0,3905.0,3905.0,505611.783019,502059.557636,497218.028527
2,1450,2023-12-31,742000.0,773000.0,746000.0,689999.0,709000.0,749900.0,315.487572,332.876186,...,10857.0,10857.0,10857.0,10857.0,4414.0,4414.0,4414.0,721077.243045,717990.323500,713885.865692
3,1460,2023-12-31,670000.0,721000.0,832500.0,659900.0,669000.0,669000.0,340.852074,360.105581,...,9848.0,9848.0,9848.0,9848.0,4248.0,4248.0,4248.0,719168.891707,715127.941197,709252.142005
4,1463,2023-12-31,520500.0,500000.0,577950.0,512000.0,475000.0,497000.0,255.043228,251.245743,...,11253.0,11253.0,11253.0,11253.0,5181.0,5181.0,5181.0,533010.208454,530562.867366,526301.335629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6221,98682,2023-12-31,476000.0,499950.0,520000.0,509248.0,499800.0,524850.0,290.739411,290.922908,...,64762.0,64762.0,64762.0,64762.0,28631.0,28631.0,28631.0,484977.406040,484497.088539,483387.291369
6222,98683,2023-12-31,565000.0,561000.0,558500.0,539900.0,550000.0,549450.0,317.966903,311.501651,...,30562.0,30562.0,30562.0,30562.0,11836.0,11836.0,11836.0,545484.547891,544705.733566,543041.038042
6223,98684,2023-12-31,459900.0,472750.0,514950.0,459900.0,455000.0,485000.0,285.714286,273.427961,...,35711.0,35711.0,35711.0,35711.0,15892.0,15892.0,15892.0,475164.711361,475130.703842,474267.556700
6224,98685,2023-12-31,640000.0,640000.0,605000.0,599000.0,599475.0,648000.0,291.176471,297.304440,...,29456.0,29456.0,29456.0,29456.0,11629.0,11629.0,11629.0,577994.879622,577247.107610,575506.070264


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
import xgboost as xgb

import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor


: 

In [ ]:
# Step 6: Min-Max scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X)
X_predict_scaled = scaler.transform(df_predict)


In [ ]:

# Step 7: Train XGBoost classifier
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)


In [ ]:
xgb_model.fit(X_train_scaled, y)

# Step 8: Predict and evaluate
y_pred = xgb_model.predict(X_predict_scaled)

'''# Step 9: Evaluation
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))'''

In [ ]:
import os
from datetime import datetime
import joblib

In [ ]:
# Define model save path
model_dir = "/home/jessi/code/JensKlug/zillows_real_estate/zillow/model/models"
os.makedirs(model_dir, exist_ok=True)

# Timestamp for versioning
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = os.path.join(model_dir, f"{timestamp}_xgboost_model.joblib")

# Save pipeline
joblib.dump(xgb_model, model_path)

print(f"\n✅ Model saved to: {model_path}")

In [ ]:
#df_train
df_predict

In [ ]:
grouped_train = df_train.groupby(['zipcode'])
grouped_predict = df_predict.groupby(['zipcode'])

In [ ]:
ts_train = pd.DataFrame()

ls = [1,3,6,12]

i = 1

#for i in ls:
ts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * df_train['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_train['zipcode_obj'] = grouped_train['zipcode'].shift(0).astype('object') # but remains still an integer...
numeric_column_names = df_train.select_dtypes(include='number').columns # drop the lags of zipcode-> this in Jessica's code.


for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

ts_train = ts_train.drop(columns=['zipcode_1m', 'zipcode_2m', 'zipcode_3m'])

In [ ]:
ts_train

In [ ]:
ts_test = pd.DataFrame()

#for i in ls:
ts_test[f'target_profitable_{i}m'] = (grouped_predict['price'].shift(-i) > 1.02 * df_predict['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_test['zipcode'] = grouped_predict['zipcode'].shift(0)# .astype('object') # but remains still an integer...
numeric_column_names = df_predict.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_test[f'{feature}_{lag}m'] = grouped_predict[f'{feature}'].shift(lag)

In [ ]:
ts_test